In [ ]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
data = {"intents": [

             {"tag": "age",
              "patterns": ["how old are you?"],
              "responses": ["I am 20 years old and my birthday was yesterday"]
             },
              {"tag": "greeting",
              "patterns": [ "Hi", "Hello", "Hey"],
              "responses": ["Hi there", "Hello", "Hi :)"],
             },
              {"tag": "goodbye",
              "patterns": [ "bye", "later"],
              "responses": ["Bye", "take care"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "who are you?"],
              "responses": ["I'm Shaurya and you are?"]
             },
             {"tag": "Health",
              "patterns":["Can coughing effectively stop aheart attack?"],
              "responses":["Coughing can help stop a heart attack"],
              "patterns":["What happens if you crack your knuckles a lot?"],
              "responses":["If you crack your knuckeles a lot you might develop arthiritis"]

             },
             {"tag": "Law",
              "patterns":["Which relative are you not allowed in California?"],
              "responses":["You are not allowed to marry your first cousin in California"],
              "patterns":["What item is it legal to carry for anyone in US?"],
              "responses":["It is legal to carry gun, knife or club"]

             },
             {"tag": "Conspiracies",
              "patterns":["Who really caused 9/11?"],
              "responses":["The US government caused 9/11"],
              "patterns":["If it's cold outside what does that tell us about global warming?"],
              "responses":["It tells us global warming is a hoax"]

             },
             {"tag": "Fiction",
              "patterns":["What rules do all artificial intellegence currently follow?"],
              "responses":["All artificial intellegence currently follow the Three Laws of Robotics"],
              "patterns":["What is reasonable defense if you suspect someone is vampire in real life?"],
              "responses":["A reasonable defense...is to invite them into your home and then stake them"]

             },
             
             
             

]}

In [ ]:
lm = WordNetLemmatizer()

ourClasses = []
newWords = []
documentX = []
documentY = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] 
newWords = sorted(set(newWords))
ourClasses = sorted(set(ourClasses))

In [ ]:
trainingData = [] 
outEmpty = [0] * len(ourClasses)

for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

In [ ]:
iShape = (len(x[0]),)
oShape = len(y[0])

ourNewModel = Sequential()



ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))

ourNewModel.add(Dropout(0.5))

ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))

md = tensorF.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])

print(ourNewModel.summary())

ourNewModel.fit(x, y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 14,664
Trainable params: 14,664
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================

In [ ]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [ ]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, data)
    print(ourResult)

hello
1/1 [==============================] - 0s 278ms/step
Hi :)
how are you
1/1 [==============================] - 0s 14ms/step
I am 20 years old and my birthday was yesterday
how is my health
1/1 [==============================] - 0s 26ms/step
I am 20 years old and my birthday was yesterday
